In [1]:
import pandas as pd
from glob import glob
from pathlib import Path
import cv2
import os
import random

In [2]:
#global variables, name option still to be implemented

batch_name = 'B0001480'
batch_path = '../../data/1930_census/Gotland/B0001480'
batch_write_path = os.path.join('../../raw/1930_census/name/gotland/original', batch_name)
dataset_type = 'name'

In [3]:
db_gotland = pd.read_csv('../../data/1930_census/Gotland/Folkräkning_1930_Gotlands_län', '\t', dtype=str)
db_multi = db_gotland.set_index(['BILDID', 'RAD']).sort_index()
#db_multi.index.names
batch_filter = pd.read_excel('../../data/1930_census/Gotland/B0001480/B0001480_filter.xlsx', dtype=str)

In [4]:
#change one digit filename to twodigit filename for the files to be sorted correctly

def correct_filename(page_path):
    
    img_paths = glob(page_path + '/**')
    
    corr_imgs = []

    for img in img_paths:
        suffix = img.split('/')[-1].split('.')[0].zfill(2) + '.jpg'
        li = img.split('/')
        li.pop()
        p = '/'.join(li) + '/' + suffix
        corr_imgs.append(p)

    corr_imgs.sort()

    return corr_imgs

In [5]:
def write_page_imgs(page_path, dataset_type, name, db, batch_filter, batch_path, batch_write_path, binarize = True):
    
    
    
    gt_file_path = os.path.join(batch_write_path, batch_name + '_gt.txt')
    
    with open(gt_file_path, 'a') as f:

        page_write_path = os.path.join(batch_write_path, name)
        
        os.mkdir(page_write_path)

        img_paths = correct_filename(page_path)

        skip_imgs = []
        
        if dataset_type == 'year':
            skip_imgs = batch_filter.skip_img_year.values
        elif dataset_type == 'name':
            skip_imgs = batch_filter.skip_img_name.values


        for i, p in enumerate(img_paths):

            row = str(i + 1)

            if (name + '_' + row) in skip_imgs:
                print(p)
                continue 

            if (name, row) in db.index:
            
                res = db.loc[name, row]

            else:
                res = None
            
            #res = db.query('BILDID==@name & RAD==@row')

            
            
            if res is not None and len(res) == 1:

                if dataset_type == 'name':
                
                    fam = res['FAMST'].item() if not pd.isnull(res['FAMST'].item()) else ''
                    prenames = res['FORNAMN'].item() if not pd.isnull(res['FORNAMN'].item()) else ''
                    surnames = res['ENAMN'].item() if not pd.isnull(res['ENAMN'].item()) else ''
                    if surnames != '':
                        continue
                    nameslist = [fam, prenames]
                    nameslist = [x + ' ' if x != '' else x for x in nameslist]
                    name_comp = ''.join(nameslist)
                    name_comp = name_comp.strip()
                    
                    if name_comp != '':

                        file_name = name + '_' + row.zfill(2) + '.jpg'
                        
                        f.write(file_name + '\t' + name_comp + '\n')

                elif dataset_type == 'year':
                
                    if not pd.isnull(res['FODAR'].item()):
                        year = res['FODAR'].item()[2:4]

                        file_name = name + '_' + row.zfill(2) + '.jpg'
                        
                        f.write(file_name + '\t' + year + '\n')

                img_p = os.path.join(page_path, row + '.jpg')

                if binarize:
                    img = cv2.imread(img_p)
                    dst = cv2.fastNlMeansDenoising(img, h=31, templateWindowSize=7, searchWindowSize=21)
                    img_grey = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
                    img_blur = cv2.medianBlur(img_grey,3).astype('uint8')
                    img_thresh_Gaussian = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

                    img_write_path = os.path.join(page_write_path, file_name)

                    cv2.imwrite(img_write_path, img_thresh_Gaussian)

                else:
                    img = cv2.imread(img_p, cv2.IMREAD_GRAYSCALE)
                    img_write_path = os.path.join(page_write_path, file_name)

                    cv2.imwrite(img_write_path, img)

                #print(surname)
                    #f.write('\n' + res['ENAMN'].item())

                
                
            





In [6]:
%%time

os.mkdir(batch_write_path)

p_stand = os.path.join(batch_path, batch_name + '_standard')
p_spec = os.path.join(batch_path, batch_name + '_crop_special')

stand = glob(p_stand + '/**')
spec = glob(p_spec + '/**')

stand = [x.split('/')[-1] for x in stand]
spec = [x.split('/')[-1] for x in spec]

stand.sort()
spec.sort()

#hack to account for pages existing in standrad but not crop_special and vice versa

use_stand = []
use_spec = []

for p in spec:
    for i, x in enumerate(stand):
        if p == x:
            break
        elif i == len(stand) - 1:
            use_spec.append(p)

for p in stand:
    for i, x in enumerate(spec):
        if p == x:
            break
        elif i == len(spec) - 1:
            use_stand.append(p)

pages = list(set(spec + stand))

pages.sort()

for i, page in enumerate(pages):

    print(str(i))
    
    if (page in use_spec) and page not in batch_filter.skip_page_name.values:
        
        page_path = os.path.join(batch_path, batch_name + '_' + 'crop_special', page, dataset_type)
        
        write_page_imgs(page_path = page_path,
                        dataset_type = dataset_type,
                        name = page,
                        db = db_multi,
                        batch_filter = batch_filter,
                        batch_path = batch_path,
                        batch_write_path=batch_write_path,
                        binarize=False)

    elif (page in use_stand) and page not in batch_filter.skip_page_name.values:
        
        page_path = os.path.join(batch_path, batch_name + '_' + 'standard', page, dataset_type)
        
        write_page_imgs(page_path = page_path,
                        dataset_type = dataset_type,
                        name = page,
                        db = db_multi,
                        batch_filter = batch_filter,
                        batch_path = batch_path,
                        batch_write_path=batch_write_path,
                        binarize=False)

    elif page not in batch_filter.skip_page_name.values:
        coin = random.randint(0, 1)
        
        if coin == 0:
            
            page_path = os.path.join(batch_path, batch_name + '_' + 'crop_special', page, dataset_type)
            
            write_page_imgs(page_path = page_path,
                            dataset_type = dataset_type,
                            name = page,
                            db = db_multi,
                            batch_filter = batch_filter,
                            batch_path = batch_path,
                            batch_write_path=batch_write_path,
                            binarize=False)

        else:

            page_path = os.path.join(batch_path, batch_name + '_' + 'standard', page, dataset_type)

            write_page_imgs(page_path = page_path,
                            dataset_type = dataset_type,
                            name = page,
                            db = db_multi,
                            batch_filter = batch_filter,
                            batch_path = batch_path,
                            batch_write_path=batch_write_path,
                            binarize=False)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
print(batch_filter.skip_img_year.values[0])

B0001097_00311_26


In [ ]:
import glob
from pathlib import Path 

imgs = glob.glob('/content/drive/MyDrive/Riksarkivet/htr/data/CycleGAN_Datasets/only_overwritten/testA/*.jpg')
imgs.sort()


for i, im in enumerate(imgs):
  img = cv2.imread(im)
  
  dst = cv2.fastNlMeansDenoising(img, h=31, templateWindowSize=7, searchWindowSize=21)
  img_grey = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
  img_blur = cv2.medianBlur(img_grey,3).astype('uint8')
  img_thresh_Gaussian = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

  cv2.imwrite('/content/drive/MyDrive/Riksarkivet/htr/CycleGAN_models/Only_overwritten/only_cv2/test_%s.jpg' % str(i).zfill(2), img_thresh_Gaussian)

In [ ]:
remove_zero

In [30]:
testp = '../../../data/1930_census/Gotland/B0001097/B0001097_crop_special/B0001097_00006/year'
imgs = glob(testp + '/**')

corr_list = []
for img in imgs:
    suffix = img.split('/')[-1].split('.')[0].zfill(2) + '.jpg'
    li = img.split('/')
    li.pop()
    p = '/'.join(li) + '/' + suffix
    corr_list.append(p)

corr_list.sort()

#imgs = [x.zfill(5) for x in imgs]